In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv('https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/storepurchasedata_large.csv')

In [3]:
dataset.describe()

,Age,Salary,Purchased
count,1554.000000,1554.000000,1554.000000
mean,44.296010,57042.471042,0.694981
std,17.462458,21209.244800,0.460564
min,18.000000,20000.000000,0.000000
25%,27.000000,46000.000000,0.000000
50%,43.000000,60000.000000,1.000000
75%,62.000000,66000.000000,1.000000
max,69.000000,96000.000000,1.000000


In [4]:
dataset.head()

,Age,Salary,Purchased
0,18,20000,0
1,19,22000,0
2,20,24000,0
3,21,28000,0
4,22,60000,1


In [5]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:,-1].values

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.20,random_state=0)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
Xtrain_ = torch.from_numpy(X_train).float()
Xtest_ = torch.from_numpy(X_test).float()

In [9]:
Xtrain_

tensor([[-0.0935,  0.2215],
        [ 1.2217, -0.5342],
        [-1.0655,  0.4104],
        ...,
        [ 1.2789,  0.4104],
        [-0.9512,  0.2215],
        [-1.2943, -1.4316]])

In [10]:
ytrain_ = torch.from_numpy(y_train)
ytest_ = torch.from_numpy(y_test)

In [11]:
ytrain_

tensor([1, 1, 1,  ..., 1, 1, 0])

In [12]:
Xtrain_.shape, ytrain_.shape

(torch.Size([1243, 2]), torch.Size([1243]))

In [13]:
Xtest_.shape, ytest_.shape

(torch.Size([311, 2]), torch.Size([311]))

In [14]:
input_size=2
output_size=2
hidden_size=10

In [15]:
class Net(nn.Module):
   def __init__(self):
       super(Net, self).__init__()
       self.fc1 = torch.nn.Linear(input_size, hidden_size)
       self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
       self.fc3 = torch.nn.Linear(hidden_size, output_size)


   def forward(self, X):
       X = torch.relu((self.fc1(X)))
       X = torch.relu((self.fc2(X)))
       X = self.fc3(X)

       return F.log_softmax(X,dim=1)

In [16]:
model = Net()



In [17]:
import torch.optim as optim
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.NLLLoss()

In [18]:
epochs = 100


In [19]:
for epoch in range(epochs):
  optimizer.zero_grad()
  Ypred = model(Xtrain_)
  loss = loss_fn(Ypred,  ytrain_)
  loss.backward()
  optimizer.step()
  print('Epoch',epoch, 'loss',loss.item())

Epoch 0 loss 0.6417384743690491
Epoch 1 loss 0.6259592771530151
Epoch 2 loss 0.6108320355415344
Epoch 3 loss 0.5958574414253235
Epoch 4 loss 0.5803265571594238
Epoch 5 loss 0.5645008683204651
Epoch 6 loss 0.5484357476234436
Epoch 7 loss 0.5320326089859009
Epoch 8 loss 0.515433132648468
Epoch 9 loss 0.4983823001384735
Epoch 10 loss 0.48086777329444885
Epoch 11 loss 0.4632143974304199
Epoch 12 loss 0.4455576241016388
Epoch 13 loss 0.4273105263710022
Epoch 14 loss 0.40881675481796265
Epoch 15 loss 0.3903775215148926
Epoch 16 loss 0.3721381723880768
Epoch 17 loss 0.3539782762527466
Epoch 18 loss 0.33601534366607666
Epoch 19 loss 0.3186277747154236
Epoch 20 loss 0.3016931116580963
Epoch 21 loss 0.28515511751174927
Epoch 22 loss 0.26933759450912476
Epoch 23 loss 0.2543560564517975
Epoch 24 loss 0.2404409795999527
Epoch 25 loss 0.2274823933839798
Epoch 26 loss 0.2153061032295227
Epoch 27 loss 0.2039642632007599
Epoch 28 loss 0.19339443743228912
Epoch 29 loss 0.18365614116191864
Epoch 30 loss 

In [20]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.1032,  1.1995],
         [ 0.4185,  0.2243],
         [-0.3409,  0.7900],
         [ 0.9989, -0.3686],
         [ 0.6399,  0.6053],
         [-1.0020, -0.9214],
         [ 0.8284,  0.1874],
         [-0.7560, -0.7009],
         [ 0.7822, -0.1588],
         [-0.6669,  0.8685]], requires_grad=True), Parameter containing:
 tensor([-0.2918,  0.6055,  0.4231, -0.0504,  0.4200, -0.1187,  0.2359,  0.8660,
          0.0279,  0.8580], requires_grad=True), Parameter containing:
 tensor([[ 0.6633,  0.0319,  0.3376,  0.5115,  0.3604, -0.2604,  0.6490, -0.2815,
           0.1490,  0.4783],
         [-0.7030,  0.2862, -0.2994, -0.2164, -0.1599,  0.7329, -0.8376,  0.4981,
           0.0049,  0.1153],
         [ 0.0186,  0.2888, -0.1792, -0.2417, -0.0895, -0.0529,  0.2126, -0.2628,
          -0.1190, -0.0304],
         [-0.8857,  0.3014, -0.3842, -0.2541, -1.0501,  0.6219, -0.7952,  0.4075,
          -0.0938, -0.0728],
         [ 0.4733,  0.4963,  0.5244,  0.7036,  

In [21]:
torch.from_numpy(sc.transform(np.array([[40,20000]]))).float()

tensor([[-0.2650, -1.7622]])

In [22]:
y_cust_20_40000 = model(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000

tensor([[-0.0088, -4.7414]], grad_fn=<LogSoftmaxBackward>)

In [23]:
_, predicted_20_40000 = torch.max(y_cust_20_40000.data,-1)
predicted_20_40000

tensor([0])

In [24]:
y_cust_42_50000 = model(torch.from_numpy(sc.transform(np.array([[42,50000]]))).float())
y_cust_42_50000

tensor([[-0.8981, -0.5232]], grad_fn=<LogSoftmaxBackward>)

In [25]:
_, predicted_42_50000 = torch.max(y_cust_42_50000.data,-1)
predicted_42_50000

tensor([1])

Export model to ONNX format

In [26]:
sample_tensor = torch.from_numpy(sc.transform(np.array([[40,20000]]))).float()


In [27]:
torch.onnx.export(model,sample_tensor,"customer.onnx",export_params=True)


In [28]:
!ls

customer.onnx  sample_data


Load the model to TensorFlow

In [29]:
!pip install onnx


In [30]:
!pip install onnx_tf


In [31]:
import onnx
import onnx_tf


In [33]:
import tensorflow_addons

In [34]:
from onnx_tf.backend import prepare


In [35]:
onnx_model = onnx.load("customer.onnx")


In [36]:
tf_rep = prepare(onnx_model)


In [37]:
prediction = tf_rep.run((sc.transform(np.array([[42,50000]]))))


In [38]:
prediction

Outputs(_0=array([[-0.8980679, -0.5231613]], dtype=float32))